In [2]:
import librosa
import soundfile as sf

def fix_wav(input_path, output_path):
    wav, sr = librosa.load(input_path, sr=16000, mono=True)
    sf.write(output_path, wav, 16000)

fix_wav("data/침착맨1.wav", "data/침착맨1_fixed.wav")
fix_wav("data/침착+나폴리mix.wav", "data/mix_fixed.wav")


/var/folders/zx/88x5v8pj1h3g803rndft70xr0000gn/T/ipykernel_2810/2177184664.py:5: UserWarning: PySoundFile failed. Trying audioread instead.
  wav, sr = librosa.load(input_path, sr=16000, mono=True)
